In [ ]:
import tensorflow as tf
from tensorflow.keras.applications import resnet50, xception, inception_v3  
from keras import Input, Model, layers

In [ ]:
import pathlib

BATCH_SIZE = 32
IMG_SIZE = (224, 224)

data_dir = pathlib.Path('drive/My Drive/TIES4911/wonders_world/Wonders of World/Wonders of World')
(train_ds, val_ds) = tf.keras.utils.image_dataset_from_directory(
    data_dir,
    shuffle=True,
    validation_split=0.2,
    subset='both',
    image_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    seed=123,
    label_mode='categorical'
)

class_names = train_ds.class_names
num_classes = len(class_names)

Found 3846 files belonging to 12 classes.
Using 3077 files for training.
Using 769 files for validation.


In [ ]:
AUTOTUNE = tf.data.AUTOTUNE

train_ds = train_ds.prefetch(buffer_size=AUTOTUNE)
val_ds = val_ds.prefetch(buffer_size=AUTOTUNE)

In [ ]:
data_augmentation = tf.keras.Sequential([
  tf.keras.layers.RandomFlip('horizontal'),
  tf.keras.layers.RandomRotation(0.2),
])

In [ ]:
inputs = Input(shape=(224, 224, 3))

base_model = xception.Xception(input_tensor=inputs, include_top=False)
base_model.trainable = False

preprocess_input = xception.preprocess_input
global_average_layer = layers.GlobalAveragePooling2D()
prediction_layer = layers.Dense(12)

x = data_augmentation(inputs)
x = preprocess_input(x)
x = base_model(x, training=False)
x = global_average_layer(x)
x = layers.Dropout(0.5)(x)
outputs = prediction_layer(x)

model = Model(inputs, outputs)
model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

checkpoint = tf.train.Checkpoint(model=model)
model.summary()

loss0, accuracy0 = model.evaluate(val_ds)

83683744/83683744 [==============================] - 1s 0us/step


Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 224, 224, 3)]     0         
                                                                 
 sequential (Sequential)     (None, 224, 224, 3)       0         
                                                                 
 tf.math.truediv (TFOpLambda  (None, 224, 224, 3)      0         
 )                                                               
                                                                 
 tf.math.subtract (TFOpLambd  (None, 224, 224, 3)      0         
 a)                                                              
                                                                 
 xception (Functional)       (None, 7, 7, 2048)        20861480  
                                                                 
 global_average_pooling2d (G  (None, 2048)             0     

In [ ]:
print(len(model.trainable_variables))

2


In [ ]:
checkpoint_path = 'drive/My Drive/TIES4911/task4/task4-2/xception-{epoch:04d}'

model_checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_path,
    save_weights_only=True,
    save_freq=2*97,
    verbose=1)

history = model.fit(train_ds,
                    epochs=10,
                    validation_data=val_ds,
                    callbacks=[model_checkpoint_callback])

Epoch 1/10


97/97 [==============================] - 642s 6s/step - loss: 9.3550 - accuracy: 0.1261 - val_loss: 10.0418 - val_accuracy: 0.1469
Epoch 2/10
96/97 [============================>.] - ETA: 4s - loss: 10.6606 - accuracy: 0.1445
Epoch 2: saving model to drive/My Drive/TIES4911/task4/task4-2/xception-0002
97/97 [==============================] - 575s 6s/step - loss: 10.6694 - accuracy: 0.1449 - val_loss: 12.4920 - val_accuracy: 0.1847
Epoch 3/10
97/97 [==============================] - 565s 6s/step - loss: 11.2464 - accuracy: 0.1518 - val_loss: 12.8064 - val_accuracy: 0.1847
Epoch 4/10
96/97 [============================>.] - ETA: 4s - loss: 11.1583 - accuracy: 0.1569
Epoch 4: saving model to drive/My Drive/TIES4911/task4/task4-2/xception-0004
97/97 [==============================] - 573s 6s/step - loss: 11.1612 - accuracy: 0.1573 - val_loss: 12.4711 - val_accuracy: 0.2016
Epoch 5/10
97/97 [==============================] - 591s 6s/step - loss: 10.9926 - accuracy: 0.1547 - val_loss: 13.016

In [ ]:
loss1, accuracy1 = model.evaluate(val_ds)

25/25 [==============================] - 111s 4s/step - loss: 13.0789 - accuracy: 0.1834
